In [1]:
import numpy as np

In [2]:
# We first define the funtion to read input files
# We read .txt file and put the data into a 2D-numpy array

def read_input(file_name):
    A = np.zeros((9,9))
    f = open(file_name,'r')
    for row in range(9):
        line = f.readline()
        ls = line.split()
        for column in range(9):
            A[row][column] = int(ls[column])
    f.close()
    return A

In [3]:
# This function checks completeness of the task

def checkCompleteness(A):
    for i in range(9):
        for j in range(9):
            if A[i][j] == 0:
                return False
    return True

In [4]:
# This function checks whether the assignment of the position(row,column) wirh value violates consistency
# return True if it's consistent and return False if it's not

def checkConsistency(A, row, column, value):
    # First check the column consistency
    for i in range(9):
        if i != row and A[i][column] == value:
            return False
    # Second check the row consistency
    for j in range(9):
        if j != column and A[row][j] == value:
            return False  
    
    # Then we check the consistency in the 3x3 box the target positin in
    # The following sub-function checks whether the value in (target_row,target_col) is used 
    # in the corresponding box in A where the corresponding box is the box
    # with position(row,col) to be the top-left corner of the 3x3 box
    def box_invalid(A, row, col, target_row, target_col, value):
        for i in range(3):
            for j in range(3):
                if not ((i+row) == target_row and (j+col) == target_col):
                    if A[i+row][j+col] == value:
                        return True
        return False
    
    boxInvalid = box_invalid(A, row - row % 3, column - column % 3, row, column, value)   
    if boxInvalid:
        return False
    
    return True 

In [5]:
# Degree Heuristic
# This function returns a 9x9 2D-array with the value in each position showing the 
# number of unassigned neighbours of each variable
# For convenience, we assign 0 to all the assigned variables

def degree(A):
    M = np.zeros((9,9))
    for i in range(9):
        for j in range(9):
            if A[i][j] != 0:
                continue
            else:
                M[i][j] = (8-np.count_nonzero(A[i])) + (8-np.count_nonzero(A.T[j])) + other_zeros_in_box(i,j,A)
    return M

def other_zeros_in_box(i,j,A):
    listi = [0,1,2]
    listj = [0,1,2]
    listi.remove(i%3)
    listj.remove(j%3)
    count = 0
    for ii in listi:
        for jj in listj:
            if A[(i//3)*3 + ii][(j//3)*3 + jj] == 0:
                count += 1
    return count

In [6]:
# Minimum Remaining Value Heuristic
# This function returns a 9x9 2D-array M with the value in each position
# showing the number of possible value assignments of this position
# For convenience, we assign 10 to all the assigned variables

def MRV(A):
    M = np.zeros((9,9)) 
    for i in range(9):
        for j in range(9):
            if A[i][j] != 0:
                M[i][j] = 10
            else:
                count = 0
                for x in range(1,10):
                    if checkConsistency(A, i, j, x):
                        count += 1
                    else:
                        continue
                M[i][j] = count             
    return M

In [7]:
# This function applies the Minimum Remaining Value and Degree Heuristic functions defined above
# Return a position to be the next variable to be assigned

def SelectUnassignedVariable(A):
    M = MRV(A)
    MRV_result = np.where(M == np.amin(M))
    listOfCordinates = list(zip(MRV_result[0], MRV_result[1]))
    # MVR_position gives a list of positions share the Minimum Remaining Value
    MVR_position = [cord for cord in listOfCordinates]
    
    D = degree(A)
    for i in range(9):
        for j in range(9):
            if (i,j) not in MVR_position:
                D[i][j] = 0
                
    if np.count_nonzero(D) == 0:
        return MVR_position

    D_result = np.where(D == np.amax(D))
    listOfCordinates2 = list(zip(D_result[0], D_result[1]))
    final_positions = [cord for cord in listOfCordinates2]
    
    # final_positions record all the possible next variables to be chosen after applying MRV and degree heuristic
    return final_positions

In [8]:
# Making use of the utility functions defined above
# we now implement the main Backtracking function to solve Sudoku

def Backtracking(A):
    if checkCompleteness(A):
        return True
    else:
#         positions = SelectUnassignedVariable(A)
#         for position in positions:
#             i, j = position
        i,j = SelectUnassignedVariable(A)[0]
        for value_index in range(9):
            if checkConsistency(A, i, j, value_index + 1):
                A[i][j] = value_index + 1
                if Backtracking(A):
                    return True
            A[i][j] = 0
        return False   

In [9]:
# This function reads our filled Sudoku and writes it in a output .txt file

def write_output(file_name, A):
    f = open(file_name, 'w')
    for row in range(9):
        line = ' '.join(str(int(i)) for i in A[row])
        f.write(line)
        if row != 8:
            f.write('\n')
    f.close()   

In [10]:
# Now we execute our program on the three tasks

def main():
    input_files = ['Input1.txt','Input2.txt','Input3.txt']
    output_files = ['Output1.txt','Output2.txt','Output3.txt']
    for i in range(3):
        Sudoku = read_input(input_files[i])
        Backtracking(Sudoku)
        write_output(output_files[i], Sudoku)
        
main()